In [1]:
import pandas as pd
import numpy as np

#sur train.redux + test.redux!!!
#df = pd.read_csv('~/Downloads/train_concat_95_00001.csv')
df = pd.read_csv('E:\\Downloads\\global_redux.csv')
#df = pd.read_csv('C:\\Users\\Paul\\Downloads\\global_redux.csv')
df.apply(pd.to_numeric,downcast='unsigned')
#y = pd.read_csv('~/Downloads/y.csv', sep=';', usecols=[1])
#y.apply(pd.to_numeric,downcast='unsigned')

def Hardy_Weinberg_Equilibrium_exact_test(obs_hets, obs_hom1=0, obs_hom2=0):
    #<param name="obs_hets" type="eval" value="" label="Observed heterozygosity: "/>
    #<param name="obs_hom1" type="eval" value="" label="Observed AA homozygosity: "/>
    #<param name="obs_hom2" type="eval" value="" label="Observed aa homozygosity: "/>
        if obs_hom1 < 0 or obs_hom2 < 0 or obs_hets < 0:
                raise Exception("FATAL ERROR - SNP-HWE: Current genotype configuration (%s  %s %s) includes negative count" % (obs_hets, obs_hom1, obs_hom2))

        obs_homc = obs_hom2 if obs_hom1 < obs_hom2 else obs_hom1
        obs_homr = obs_hom1 if obs_hom1 < obs_hom2 else obs_hom2

        rare_copies = 2 * obs_homr + obs_hets
        genotypes   = obs_hets + obs_homc + obs_homr

        het_probs = [0.0] * (rare_copies + 1)

        #start at midpoint
        mid = int(rare_copies * (2 * genotypes - rare_copies) / (2 * genotypes))

        #check to ensure that midpoint and rare alleles have same parity
        if (rare_copies & 1) ^ (mid & 1):
                mid += 1

        curr_hets = mid
        curr_homr = (rare_copies - mid) / 2
        curr_homc = genotypes - curr_hets - curr_homr

        het_probs[mid] = 1.0
        sum = float(het_probs[mid])

        for curr_hets in range(mid, 1, -2):

                het_probs[curr_hets - 2] = het_probs[curr_hets] * curr_hets * (curr_hets - 1.0) / (4.0 * (curr_homr + 1.0) * (curr_homc + 1.0))

                sum += het_probs[curr_hets - 2];

                # 2 fewer heterozygotes for next iteration -> add one rare, one common homozygote
                curr_homr += 1
                curr_homc += 1

        curr_hets = mid
        curr_homr = (rare_copies - mid) / 2
        curr_homc = genotypes - curr_hets - curr_homr

        for curr_hets in range(mid, rare_copies - 1, 2):

                het_probs[curr_hets + 2] = het_probs[curr_hets] * 4.0 * curr_homr * curr_homc / ((curr_hets + 2.0) * (curr_hets + 1.0))

                sum += het_probs[curr_hets + 2]

                #add 2 heterozygotes for next iteration -> subtract one rare, one common homozygote
                curr_homr -= 1
                curr_homc -= 1

        for i in range(0, rare_copies + 1):
                het_probs[i] /= sum

        #alternate p-value calculation for p_hi/p_lo
        p_hi = float(het_probs[obs_hets])
        for i in range(obs_hets, rare_copies+1):
                p_hi += het_probs[i]

        p_lo = float(het_probs[obs_hets])
        for i in range(obs_hets-1, -1, -1):
                p_lo += het_probs[i]

        p_hi_lo = 2.0 * p_hi if p_hi < p_lo else 2.0 * p_lo

        p_hwe = 0.0
        #  p-value calculation for p_hwe
        for i in range(0, rare_copies + 1):
                if het_probs[i] > het_probs[obs_hets]:
                        continue;
                p_hwe += het_probs[i]

        p_hwe = 1.0 if p_hwe > 1.0 else p_hwe

        return p_hwe
    
def uni1():
	if int(10000 * Hardy_Weinberg_Equilibrium_exact_test(762, 269, 618)) != 1973:
		return "Failed to compute HWE of 762, 269, 618"

	return True

In [49]:
totalRows = df.shape[0]
MAF = pd.DataFrame(columns={'SNP', 'maf'})
HWE = pd.DataFrame(columns={'SNP', 'hwe'})
SNP = pd.DataFrame(columns={'SNP'})
SNP['SNP'] = df.columns.values.ravel()
lst_hwe = []
lst_maf = []

for i in range(df.shape[1]):
    total = df.iloc[:, i].value_counts()
    
    #Hardy-Weinberg equilibrium (HWE) < 0.000001
    if total.shape[0] > 2:
        totmut = total[2]
    else:
        totmut = 0
    hwe = float(Hardy_Weinberg_Equilibrium_exact_test(total[1], total[0], totmut))
    lst_hwe.append(hwe)
        
    #MAF => - 0.05 d'expression d'une mutation (2 a 5% conseille)
    lst_maf.append(1 - (total[0] / totalRows))

HWE['SNP'] = SNP
HWE['hwe'] = lst_hwe
MAF['SNP'] = SNP
MAF['maf'] = lst_maf
HWE.to_csv('E:\\Downloads\\QC_HWE.csv', index=False)
MAF.to_csv('E:\\Downloads\\QC_MAF.csv', index=False)
print('Total SNP: ' + str(df.shape[1]))

Total SNP: 18124


In [50]:
#HWE = pd.read_csv('E:\\Downloads\\QC_HWE.csv')
threshold = 0.000001
lowHWE = HWE[HWE.hwe < threshold]
name = 'E:\\Downloads\\QC_lowHWE_' + str(threshold) + '.csv'
lowHWE.to_csv(name , index=False)
lowHWE.shape

(1990, 2)

In [51]:
#MAF = pd.read_csv('E:\\Downloads\\QC_MAF.csv')
threshold = 0.05
lowMAF = MAF[MAF.maf < threshold]
name = 'E:\\Downloads\\QC_lowMAF_' + str(threshold) + '.csv'
lowMAF.to_csv(name , index=False)
lowMAF.shape

(138, 2)

In [54]:
import pandas as pd
import numpy as np

#t1 = pd.read_csv('C:\\Users\\Paul\\Downloads\\QC_lowMAF_05.csv', names=['a','b'])
t1 = pd.read_csv('E:\\Downloads\\QC_lowMAF_0.05.csv', names=['a','b'])
t1.drop(['b'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
#t2 = pd.read_csv('C:\\Users\\Paul\\Downloads\\QC_lowHWE_000001.csv', names=['a','b'])
t2 = pd.read_csv('E:\\Downloads\\QC_lowHWE_1e-06.csv', names=['a','b'])
t2.drop(['b'], axis=1, inplace=True)
t2 = t2.drop(t2.index[:1])
concatenated = pd.concat([t1, t2], axis=0)
concatenated.drop_duplicates('a')
print(concatenated.shape)

(2128, 1)


In [55]:
import pandas as pd
import numpy as np

df = pd.read_csv('E:\\Downloads\\train_redux.csv')
#df = pd.read_csv('C:\\Users\\Paul\\Downloads\\global_redux.csv')
df.apply(pd.to_numeric,downcast='unsigned')
export = df.drop(concatenated['a'].tolist(), axis=1)
name = 'E:\\Downloads\\train_redux_QC_05_000001.csv'
export.to_csv(name, index=False)
export.shape

(26500, 16004)